一. tf.kears与keras的区别

    keras以Tensorflow、cntk或者Theano为后端运行，keras必须有后端才可以运行，后端可以切换，现在多用tensorflow. 
    tf.keras对keras API规范的实现，相对于以tensorflow为后端的keras，tf.keras与tensorflow结合更加紧密.
    联系：
       1. 基于同一套API，keras可以通过改导入方式轻松转化为tf.keras程序，反之不成立，因为tf.keras有其它特性；
       2. 相同的JSON和HDF5模型序列化格式和定义
    区别：
       1. Tf.keras全面支持eager mode
           1). 只是用keras.Sequential和keras.Model时没有影响
           2). 自定义Model内部运算逻辑的时候会有影响，tf底层api可以使用keras的model.fit等抽象,tf.keras更支持开发
       2. tf.keras支持基于tf.data的模型训练；
       3. tf.keras支持tpu训练；
       4. tf.keras支持tf.distribution中的分布式策略
       5. tf.keras可以与tensorflow中的estimator集成
       6. tf.keras可以保存为SavedModel
    keras 的中文文档  https://keras.io/zh/

**二. 如何选择**

   1. 如果想用tf.keras中的任何一个特性，则使用tf.keras；
   2. 如果后端互换很重要那么使用keras；
   3. 如果都不重要那么随便选。 

**三. keras入门**

**1. 创建模型**

   Keras有两种类型的模型，序贯模型（Sequential）和函数式模型（Model），函数式模型应用更为广泛，序贯模型是函数式模型的一种特殊情况。
   Sequential models：这种方法用于实现一些简单的模型。你只需要向一些存在的模型中添加层就行了。<br/>
   Functional API：Keras的API是非常强大的，你可以利用这些API来构造更加复杂的模型，比如多输出模型，有向无环图等等。<br/>
   两类模型一些相同的方法：<br/>
      model.summary()：打印出模型概况
      model.get_config():返回包含模型配置信息的Python字典。模型也可以从它的config信息中重构回去
      model.get_layer()：依据层名或下标获得层对象
      model.get_weights()：返回模型权重张量的列表，类型为numpy array
      model.set_weights()：从numpy array里将权重载入给模型，要求数组具有与model.get_weights()相同的形状。
      model.to_json：返回代表模型的JSON字符串，仅包含网络结构，不包含权值。可以从JSON字符串中重构原模型：                
      model.save_weights(filepath)：将模型权重保存到指定路径，文件类型是HDF5（后缀是.h5）
      model.load_weights(filepath, by_name=False)：从HDF5文件中加载权重到当前模型中, 默认情况下模型的结构将保持不变。如果想将权重载入不同的模型（有些层相同）中，
                   则设置by_name=True，只有名字匹配的层才会载入权重

In [4]:
####导入模块
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.3
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [6]:
####sequential模型是层的简单堆叠，无法表示任意模型，有两种创建的方式，一种是add一层一层增加，一种直接创建
"""
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
"""

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), #将输入展平
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

model.compile(loss="binary_crossentropy",
              optimizer = "rmsprop",
              metrics = ["accuracy"])

In [7]:
model.layers

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [10]:
####使用keras函数式API可以构建复杂的模型拓扑，如：多输入所输出
input_x = tf.keras.Input(shape=(72,))
hidden1 = keras.layers.Dense(32, activation='relu')(input_x)
hidden2 = keras.layers.Dense(16, activation='relu')(hidden1)
pred = keras.layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 72)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2336      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 10)                170       
Total params: 3,034
Trainable params: 3,034
Non-trainable params: 0
_________________________________________________________________


**2. 模型子类化**

In [19]:
#通过对 tf.keras.Model 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。在 init 方法中创建层并将它们设置为类实例的属性。在 call 方法中定义前向传播
class MyModel(tf.keras.Model):
    def __init__(self,num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = keras.layers.Dense(32, activation='relu')
        self.layer2 = keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

**3. 自定义层**

In [20]:
#通过对 tf.keras.layers.Layer 进行子类化并实现以下方法来创建自定义层：
     # build：创建层的权重。使用 add_weight 方法添加权重。
     # call：定义前向传播。
     # compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。 或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。

class MyLayer(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = tf.keras.Sequential(
[
    MyLayer(10),
    keras.layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

In [22]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

**4.回调函数**

In [23]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks, validation_data=(val_x, val_y))

NameError: name 'train_x' is not defined

**5. 保存和恢复**

In [24]:
####保存权重
model = tf.keras.Sequential([
keras.layers.Dense(64, activation='relu'),
keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./model.h5')
model.load_weights('./model.h5')

####保存网络结构
# 序列化成json
import json
import pprint
json_str = model.to_json()
pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)
# 保持为yaml格式  #需要提前安装pyyaml
yaml_str = model.to_yaml()
print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

######保存整个模型
model = tf.keras.Sequential([
  keras.layers.Dense(10, activation='softmax', input_shape=(72,)),
  keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
model.save('all_model.h5')
model = tf.keras.models.load_model('all_model.h5')

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_15',
                                   'trainable': True,
                                   'units': 64,

ImportError: Requires yaml module installed (`pip install pyyaml`).

**6. 将keras用于Estimator**

In [25]:
###Estimator API 用于针对分布式环境训练模型。它适用于一些行业使用场景，例如用大型数据集进行分布式训练并导出模型以用于生产
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
estimator = tf.keras.estimator.model_to_estimator(model)

NameError: name 'layers' is not defined